# Milestone 1 solution: Loading and cleaning the data

In [1]:
# this plugin requires to run "pip install nb-black"
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd

<IPython.core.display.Javascript object>

## 1. Load the data for San-Francisco and explore a few examples

In [3]:
df = pd.read_parquet("../data/safety-SanFrancisco.parquet")

<IPython.core.display.Javascript object>

In [4]:
df.shape

(3958889, 12)

<IPython.core.display.Javascript object>

The data is in tabular form, where for each event we have a category, a subcategory, a date and a location. In total there are 3.8 million rows for San Francisco safety data.

In [5]:
df.head()

,requestId,dataType,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source,extendedProperties
0,1,Safety,911_Fire,2018-10-02 11:54:01,Potentially Life-Threatening,Medical Incident,None,ELM ST/FRANKLIN ST,37.781286,-122.422187,None,
1,2,Safety,311_All,2018-07-08 15:00:27,Street and Sidewalk Cleaning,Bulky Items,Closed,"1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",37.791887,-122.418188,Mobile/Open311,
2,3,Safety,311_All,2016-06-28 13:12:28,General Request - COUNTY CLERK,customer_callback,Closed,Not associated with a specific address,0.000000,0.000000,Phone,
3,7,Safety,311_All,2017-03-03 09:34:49,Temporary Sign Request,Temporary Sign Request for Other_Event_Type,Closed,"2190 NORTH POINT ST, SAN FRANCISCO, CA, 94123",37.802853,-122.443245,Phone,
4,10,Safety,911_Fire,2016-10-25 18:33:20,Potentially Life-Threatening,Medical Incident,None,200 Block of DALEWOOD WAY,37.737953,-122.456498,None,


<IPython.core.display.Javascript object>

Let's examine a date in particular to get a better feeling of the data, for instance, January 20, 2018. We notice however that `dateTime` columns is encoded as an object and contains string values, so we cannot use it to filter by date directly.

In [6]:
df.dtypes

requestId               int64
dataType               object
dataSubtype            object
dateTime               object
category               object
subcategory            object
status                 object
address                object
latitude              float64
longitude             float64
source                 object
extendedProperties     object
dtype: object

<IPython.core.display.Javascript object>

In [7]:
x = df["dateTime"].iloc[0]
print(x, type(x))

2018-10-02 11:54:01 <class 'str'>


<IPython.core.display.Javascript object>

We need to convert it to a `datetime` dtype in order to filter by date,

In [8]:
df["dateTime"] = pd.to_datetime(df["dateTime"])

<IPython.core.display.Javascript object>

In [9]:
mask = df["dateTime"].dt.date == pd.to_datetime("2018-01-20")
df_s = df[mask]
df_s.head()

,requestId,dataType,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source,extendedProperties
238,569,Safety,911_Fire,2018-01-20 18:15:55,Non Life-threatening,Medical Incident,None,18TH ST/HARTFORD ST,37.760955,-122.433904,None,
1047,2456,Safety,911_Fire,2018-01-20 21:49:34,Potentially Life-Threatening,Medical Incident,None,200 Block of MARKET ST,37.792824,-122.397034,None,
1508,3545,Safety,911_Fire,2018-01-20 08:29:22,Potentially Life-Threatening,Medical Incident,None,2100 Block of THOMAS AVE,37.735738,-122.400562,None,
3252,7788,Safety,311_All,2018-01-20 11:23:21,Sign Repair,Sign - Defaced,Closed,"392 GUERRERO ST, SAN FRANCISCO, CA, 94103",37.764869,-122.424401,Mobile/Open311,
4538,10853,Safety,911_Fire,2018-01-20 22:47:09,Potentially Life-Threatening,Medical Incident,None,4000 Block of 18TH ST,37.760977,-122.434457,None,


<IPython.core.display.Javascript object>

Let's check the volume of requests on this date by category and subtype to see which are most frequent,

In [10]:
df_s["dataSubtype"].value_counts()

311_All     1162
911_Fire     896
Name: dataSubtype, dtype: int64

<IPython.core.display.Javascript object>

In [11]:
df_s["category"].value_counts().head(10)

Street and Sidewalk Cleaning    420
Potentially Life-Threatening    414
Non Life-threatening            236
Graffiti                        233
Alarm                           210
Encampments                     128
Abandoned Vehicle                58
Fire                             36
MUNI Feedback                    36
Homeless Concerns                35
Name: category, dtype: int64

<IPython.core.display.Javascript object>

We can also check if there are repeated requests from the same location

In [12]:
df_s["address"].value_counts().head(10)

200 Block of MARKET ST                    31
Not associated with a specific address    22
300 Block of EDDY ST                      20
800 Block of MARKET ST                    16
CASTRO ST/MARKET ST                       13
200 Block of EDDY ST                      12
2900 Block of FOLSOM ST                   12
1300 Block of CASTRO ST                   11
1100 Block of WEBSTER ST                  10
700 Block of FRANCISCO ST                 10
Name: address, dtype: int64

<IPython.core.display.Javascript object>

For instance, what are the requests that are originating on that day from "200 Block of MARKET ST"?

In [13]:
df_s[df_s["address"] == "200 Block of MARKET ST"].sort_values("dateTime").head()

,requestId,dataType,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source,extendedProperties
585070,220100,Safety,911_Fire,2018-01-20 13:48:08,Fire,Train / Rail Incident,None,200 Block of MARKET ST,37.792679,-122.396853,None,
3281481,760186,Safety,911_Fire,2018-01-20 13:48:08,Fire,Train / Rail Incident,None,200 Block of MARKET ST,37.792679,-122.396853,None,
137243,333203,Safety,911_Fire,2018-01-20 13:48:08,Fire,Train / Rail Incident,None,200 Block of MARKET ST,37.792679,-122.396853,None,
2234616,621298,Safety,911_Fire,2018-01-20 13:48:08,Fire,Train / Rail Incident,None,200 Block of MARKET ST,37.792679,-122.396853,None,
349704,848770,Safety,911_Fire,2018-01-20 13:48:08,Fire,Train / Rail Incident,None,200 Block of MARKET ST,37.792679,-122.396853,None,


<IPython.core.display.Javascript object>

Now we notice that a number of these are identical, except for the `requestId` column. Therefore, we would likely need to run `df.drop_duplicates` to drop duplicates, but excluding the `requestId` column.

## 2. Identify potential data quality issues

We can start by checking the data type of each columns. We noticed earlier that the data type of the `dateTime` columns was incorrect, before we fixed it.

In [14]:
df.dtypes

requestId                      int64
dataType                      object
dataSubtype                   object
dateTime              datetime64[ns]
category                      object
subcategory                   object
status                        object
address                       object
latitude                     float64
longitude                    float64
source                        object
extendedProperties            object
dtype: object

<IPython.core.display.Javascript object>

The number of unique values in each columns is also relevant. Columns with less than two unique values can be discarded; here columns `dataType` and `extendedProperties`. Columns with low cardinality might be more efficiently represented as a [`pandas.Categorical` data type](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html).

In [15]:
# We compute the number of unique elements one columns at a time as it's faster
for column in df.columns:
    print(column, df[column].nunique())

requestId 1185499
dataType 1
dataSubtype 2
dateTime 2613287
category 95
subcategory 639
status 2
address 226724
latitude 1206063
longitude 1159108
source 8
extendedProperties 1


<IPython.core.display.Javascript object>

Approximately 0.1 % of requests have a missing `category` and 36% of requests have a missing `status` or `source`  entry

In [16]:
(df.isna().sum(axis=0) / df.shape[0]).round(3)

requestId             0.000
dataType              0.000
dataSubtype           0.000
dateTime              0.000
category              0.001
subcategory           0.000
status                0.355
address               0.000
latitude              0.000
longitude             0.000
source                0.355
extendedProperties    0.000
dtype: float64

<IPython.core.display.Javascript object>

For categorical columns, we notice that there is a strong imbalance of frequency. For instance, 89% of cases happen for the first 15 categories out of 90. Although not technically a data quality issue, it mean that we won't be able to accurately model categories with little volume.

In [17]:
df["category"].value_counts()

Street and Sidewalk Cleaning               965968
Potentially Life-Threatening               676297
Non Life-threatening                       349634
Alarm                                      326893
Graffiti                                   296927
                                            ...  
General Request - STATUS OF WOMEN               6
General Request - WAR MEMORIAL                  5
General Request - LANGUAGE SERVICES             5
General Request - GRANTS FOR THE ARTS           1
General Request - CONVENTION FACILITIES         1
Name: category, Length: 95, dtype: int64

<IPython.core.display.Javascript object>

In [18]:
df["category"].value_counts().head(15).sum() / df.shape[0]

0.8904162253601957

<IPython.core.display.Javascript object>

Finally, we can also automatically generate a report with some of this analysis using the [pandas-profiling](https://github.com/pandas-profiling/pandas-profiling) project. We need to subsample the data however to make report generation sufficiently fast. here we will take a random sample of 100000 requests in the data.

In [19]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df.sample(100000, random_state=0))

<IPython.core.display.Javascript object>

In [20]:
profile

Summarize dataset:   0%|          | 0/27 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

An interesting point that we haven't noticed so far, is that `latitude` and `longitude` columns, have no missing values, however they do have values set to 0, which here must have the same meaning.

In [21]:
(df[["latitude", "longitude"]] == 0.0).sum(axis=0)

latitude     109599
longitude    109599
dtype: int64

<IPython.core.display.Javascript object>

## 3. Store cleaned dataset to parquet

We we can put together everything we have learned about the data quality and write a finction that would,

 - remove columns that contain constant values (`dataType` and `extendedProperties`
 - convert `dateTime` tto a datetime type
 - convert columns that have less than a few 100 unique string values as categorical data type to use less memory.
 - removed duplicate rows

In [22]:
def clean_raw_data(df: pd.DataFrame) -> pd.DataFrame:
    """Clean the raw safety data

    Note that this function does inplace modifications of the data.
    """
    df = df.copy()
    df["dateTime"] = pd.to_datetime(df["dateTime"])
    # remove constant columns
    for col in ["dataType", "extendedProperties"]:
        del df[col]
    # convert categorical columns from string to pd.Categorical,
    # to use less memory
    for col in ["dataSubtype", "category", "subcategory", "source", "status"]:
        df[col] = pd.Categorical(df[col])

    # remove duplicate entries
    agg_columns = df.columns.tolist()
    agg_columns.remove("requestId")
    df = df.drop_duplicates(agg_columns)
    return df

<IPython.core.display.Javascript object>

In [23]:
df_p = clean_raw_data(df.copy())

<IPython.core.display.Javascript object>

the resulting cleaned data can now be stored back on disk. We will use a `-1.parquet` suffix in the filename to indicate that this data is processed.

In [24]:
df_p.to_parquet("../data/safety-SanFrancisco-1.parquet", compression="snappy")

<IPython.core.display.Javascript object>

## 4. Examine the data for New-York city

We can similarly analyze the New-York city data,

In [25]:
df_nyc = pd.read_parquet("../data/safety-Nyc.parquet")

<IPython.core.display.Javascript object>

The columns are identical,

In [26]:
print("SF", df.columns.tolist())
print("NYc", df_nyc.columns.tolist())

SF ['requestId', 'dataType', 'dataSubtype', 'dateTime', 'category', 'subcategory', 'status', 'address', 'latitude', 'longitude', 'source', 'extendedProperties']
NYc ['requestId', 'dataType', 'dataSubtype', 'dateTime', 'category', 'subcategory', 'status', 'address', 'latitude', 'longitude', 'source', 'extendedProperties']


<IPython.core.display.Javascript object>

Let's now compare categories used,

In [27]:
categories = {
    "SF": df["category"].str.lower().unique(),
    "NYc": df_nyc["category"].str.lower().unique(),
}

<IPython.core.display.Javascript object>

In [28]:
for key, val in categories.items():
    print(f"{key}: number of unique categories {len(val)}")

SF: number of unique categories 96
NYc: number of unique categories 398


<IPython.core.display.Javascript object>

In [29]:
set(categories["SF"]).intersection(categories["NYc"])

{'abandoned vehicle', 'graffiti'}

<IPython.core.display.Javascript object>

The system of categories looks very different between cities. For instance New York city has 396 categories, four times more than San-Francisco, with only 2 categories in common. Some adaptation might therefore be necessary in the future.

We can also apply the same cleaning script on all cities,

In [30]:
from pathlib import Path

data_folder = Path("../data")

for path_s in [
    "safety-Nyc.parquet",
    "safety-SanFrancisco.parquet",
]:
    path = data_folder / path_s
    print(f"Processing {path}...")
    df = pd.read_parquet(path)
    clean_raw_data(df).to_parquet(
        data_folder / path.name.replace(".parquet", "-1.parquet"),
        compression="snappy",
    )

Processing ../data/safety-Nyc.parquet...
Processing ../data/safety-SanFrancisco.parquet...


<IPython.core.display.Javascript object>